<a href="https://colab.research.google.com/github/CinthiaDayanara/modelos_emociones/blob/main/modeloentrenado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt

In [7]:
# Configuración de parámetros
IMAGE_SIZE = (224, 224)
BATCH_SIZE = 32
EPOCHS = 10
NUM_CLASSES = 5  # Cambia esto según el número de emociones en tu dataset

In [8]:
data_dir = "/content/drive/MyDrive/Train"  # Ruta a tu dataset

In [9]:
# Cargar VGG16 sin la capa de clasificación superior
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Congelar las capas del modelo base
for layer in base_model.layers:
    layer.trainable = False

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [10]:
# Añadir capas personalizadas para la clasificación de emociones
x = Flatten()(base_model.output)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(NUM_CLASSES, activation='softmax')(x)

In [11]:
# Crear modelo final
model = Model(inputs=base_model.input, outputs=x)


In [12]:

# Compilar modelo
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [13]:
# Preparar los generadores de datos
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)
train_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation'
)


Found 4803 images belonging to 5 classes.
Found 1200 images belonging to 5 classes.


In [20]:
# Entrenar el modelo con capas congeladas
history = model.fit(
    train_generator,
    epochs=EPOCHS ,
    validation_data=validation_generator
)

Epoch 1/10
151/151 ━━━━━━━━━━━━━━━━━━━━ 223s 1s/step - accuracy: 0.5987 - loss: 0.9970 - val_accuracy: 0.5317 - val_loss: 1.0914
Epoch 2/10
151/151 ━━━━━━━━━━━━━━━━━━━━ 261s 1s/step - accuracy: 0.6111 - loss: 0.9485 - val_accuracy: 0.5500 - val_loss: 1.0642
Epoch 3/10
151/151 ━━━━━━━━━━━━━━━━━━━━ 225s 1s/step - accuracy: 0.6345 - loss: 0.9209 - val_accuracy: 0.5408 - val_loss: 1.0665
Epoch 4/10
151/151 ━━━━━━━━━━━━━━━━━━━━ 227s 2s/step - accuracy: 0.6565 - loss: 0.8832 - val_accuracy: 0.5442 - val_loss: 1.0490
Epoch 5/10
151/151 ━━━━━━━━━━━━━━━━━━━━ 222s 1s/step - accuracy: 0.6607 - loss: 0.8657 - val_accuracy: 0.5758 - val_loss: 0.9925
Epoch 6/10
151/151 ━━━━━━━━━━━━━━━━━━━━ 227s 2s/step - accuracy: 0.6736 - loss: 0.8309 - val_accuracy: 0.5917 - val_loss: 0.9945
Epoch 7/10
151/151 ━━━━━━━━━━━━━━━━━━━━ 227s 2s/step - accuracy: 0.6737 - loss: 0.8181 - val_accuracy: 0.5517 - val_loss: 0.9948
Epoch 8/10
151/151 ━━━━━━━━━━━━━━━━━━━━ 232s 2s/step - accuracy: 0.6757 - loss: 0.7929 - val_accu

In [21]:
# Descongelar algunas capas del modelo base para fine-tuning
for layer in base_model.layers[-4:]:  # Ajustar cuántas capas descongelar
    layer.trainable = True


In [22]:
# Recompilar el modelo con una tasa de aprendizaje más baja
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.00001), loss='categorical_crossentropy', metrics=['accuracy'])


In [23]:
# Continuar el entrenamiento con fine-tuning
history_finetune = model.fit(
    train_generator,
    epochs=EPOCHS // 2,
    validation_data=validation_generator
)


Epoch 1/5
151/151 ━━━━━━━━━━━━━━━━━━━━ 237s 2s/step - accuracy: 0.7121 - loss: 0.7146 - val_accuracy: 0.6233 - val_loss: 0.8928
Epoch 2/5
151/151 ━━━━━━━━━━━━━━━━━━━━ 223s 1s/step - accuracy: 0.7237 - loss: 0.6884 - val_accuracy: 0.6192 - val_loss: 0.8803
Epoch 3/5
151/151 ━━━━━━━━━━━━━━━━━━━━ 219s 1s/step - accuracy: 0.7252 - loss: 0.6821 - val_accuracy: 0.6200 - val_loss: 0.9071
Epoch 4/5
151/151 ━━━━━━━━━━━━━━━━━━━━ 267s 1s/step - accuracy: 0.7448 - loss: 0.6411 - val_accuracy: 0.6308 - val_loss: 0.8743
Epoch 5/5
151/151 ━━━━━━━━━━━━━━━━━━━━ 224s 1s/step - accuracy: 0.7318 - loss: 0.6548 - val_accuracy: 0.6550 - val_loss: 0.8378


In [24]:
# Evaluar el modelo en los datos de validación
eval_results = model.evaluate(validation_generator)
print(f"Precisión en validación: {eval_results[1] * 100:.2f}%")

38/38 ━━━━━━━━━━━━━━━━━━━━ 34s 906ms/step - accuracy: 0.6360 - loss: 0.8613
Precisión en validación: 65.42%


In [26]:
# Guardar el modelo afinado
model.save("emotion_vgg16_finetuned_model.h5")